In [1]:
from nitropulse import RismaData

In [2]:
risma = RismaData(workspace_dir='/home/morteza/.nitropulse')

Local directory for RISMA data: /home/morteza/.nitropulse/inputs/RISMA_CSV_files


In [3]:
risma_stations = ['RISMA_MB1', 'RISMA_MB2', 
                  'RISMA_MB3', 'RISMA_MB4', 
                  'RISMA_MB5', 'RISMA_MB6', 
                  'RISMA_MB7', 'RISMA_MB8', 
                  'RISMA_MB9', 'RISMA_MB10', 
                  'RISMA_MB11', 'RISMA_MB12', 
                  'RISMA_MB13', 'RISMA_MB14', 
                  'RISMA_MB15', 'RISMA_MB16',
                  'RISMA_MB17', 'RISMA_MB18', 
                  'RISMA_MB19', 'RISMA_MB20',
                  'RISMA_MB21', 'RISMA_MB22', 
                  'RISMA_MB23', 'RISMA_MB24',
                  'RISMA_MB25', 'RISMA_MB26',
                  'RISMA_SK1', 'RISMA_SK2', 
                  'RISMA_SK3', 'RISMA_SK4',

                  ]
params = ['Air Temp', 'Soil temperature', 'Soil Moisture']
sensors= 'average'
depths = ['0 to 5 cm', '5 cm']

In [4]:
# risma.download_risma_data(
#     out_dir='../assets/inputs/RISMA_CSV_files', 
#     stations=risma_stations, parameters=params, 
#     sensors=sensors, depths=depths, 
#     start_date='2010-01-01', end_date='2024-01-01')

In [5]:
risma_df = risma.load_df(depth='0 to 5 cm')
print(risma_df.shape)
risma_df.head()

Found 30 files in RISMA directory.


Loading RISMA files:   0%|          | 0/30 [00:00<?, ?it/s]

Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB8.csv for station MB8
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB18.csv for station MB18
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB24.csv for station MB24
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB12.csv for station MB12
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB17.csv for station MB17
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_SK2.csv for station SK2
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB20.csv for station MB20
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB4.csv for station MB4
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_files/RISMA_MB26.csv for station MB26
Processing RISMA file: /home/morteza/.nitropulse/inputs/RISMA_CSV_file

,date,station,depth,mean_sst,mean_airt,AIRT,PRCP,SSM,SST,year,doy,Station,Depth (cm),bulk,Sand,Silt,Clay,Classification
0,2013-04-16 18:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.82,0.0,-0.160000,0.000000,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
1,2013-04-16 18:15:00,MB1,0 to 5 cm,0.108333,-1.52,-2.02,0.0,0.206667,0.050000,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
2,2013-04-16 18:30:00,MB1,0 to 5 cm,0.108333,-1.52,-1.89,0.0,0.206667,0.050000,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
3,2013-04-16 18:45:00,MB1,0 to 5 cm,0.108333,-1.52,-1.67,0.0,0.208333,0.083333,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
4,2013-04-16 19:00:00,MB1,0 to 5 cm,0.108333,-1.52,-1.72,0.0,0.208333,0.083333,2013,106,MB1,5,1.28,0.788,0.101,0.111,Sandy Loam
